In [16]:
import csv
import folium


In [23]:
# Function to compute distance between two coordinates in m
from math import sin, cos, sqrt, atan2, radians

def coord_distance(coord1, coord2):
    R = 6373.0
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c * 1000

In [24]:
def total_distance(coordinates):
    total = 0
    for i in range(1, len(coordinates)):
        total += coord_distance(coordinates[i-1], coordinates[i])
    return total

In [25]:
# Open csv file from desktop
coords = []
with open('./data/path.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    for id, line in enumerate(csv_reader):
        coords.append([float(line[3]), float(line[4]), float(line[9])])


In [26]:
temp_coords = coords.copy()
coords = [[c[0], c[1]] for c in coords] # if c[2] < 0.8]

print(f"Removed {len(temp_coords) - len(coords)} points")

Removed 0 points


In [27]:
print(f"Original: {total_distance(temp_coords)}")
print(f"Filtered: {total_distance(coords)}")

Original: 39281.26273387872
Filtered: 39281.26273387872


In [28]:
# Print coordinates on a map
m = folium.Map(location=coords[0], zoom_start=5)
for coord in coords:
    folium.Marker(coord).add_to(m)
m.save('map.html')

In [29]:
# Open map in chrome browser using terminal command
! open -a "Arc" map.html

In [106]:
# Moving window
size = 2 # Window size
cleaned = [[coords[j] for j in range(i - size, i)] for i in range(size, len(coords))]

# Compute mean point of each window
mean_points = []

# Add the original points for the first and last window size
for i in range(size//2):
    mean_points.append(coords[i])

for window in cleaned:
    mean_points.append([sum([point[0] for point in window])/size, sum([point[1] for point in window])/size])

for i in range(size//2, 0, -1):
    mean_points.append(coords[-i])

# Plot mean points on a map
m = folium.Map(location=coords[0], zoom_start=10)
for i, coord in enumerate(mean_points):
    folium.Marker(
        coord, 
        tooltip=i,
        icon=folium.Icon(color='red', icon='info-sign') if i == 37 else None
        ).add_to(m)
m.save('map.html')

# Print total distance
total_distance = 0
for i in range(1, len(mean_points)):
    total_distance += coord_distance(mean_points[i-1], mean_points[i])

print(f"Total distance: {total_distance} m")

# Open map in chrome browser using terminal command
! open -a "Arc" map.html

Total distance: 29301.39276275092 m
